In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!ls data

ls: data: No such file or directory


In [3]:
data = pd.read_csv('data/comp_predict_table.csv')
shibor_table = pd.read_csv('data/mfd_bank_shibor.csv')
yield_table = pd.read_csv('data/mfd_day_share_interest.csv')
user_balance = pd.read_csv('data/user_balance_table.csv')
user_profile = pd.read_csv('data/user_profile_table.csv')

IOError: File data/comp_predict_table.csv does not exist

In [ ]:
shibor_table.head()

In [ ]:
shibor_table.info()

In [ ]:
shibor_table.mfd_date = pd.to_datetime(shibor_table['mfd_date'], format='%Y%m%d', errors='ignore')

In [ ]:
shibor_table.head()

In [ ]:
yield_table.mfd_date = pd.to_datetime(yield_table['mfd_date'], format='%Y%m%d', errors='ignore')

In [ ]:
yield_table.head()

In [ ]:
yield_table.info()

In [ ]:
yield_table.shape

In [ ]:
user_balance.report_date = pd.to_datetime(user_balance['report_date'], format='%Y%m%d', errors='ignore')

In [ ]:
user_balance.head()

In [ ]:
user_balance.info()

In [ ]:
user_profile.head()

In [ ]:
user_profile.info()

In [ ]:
def user_type(user_type, df = user_balance):
    '''
    Classify the users as real repeat users and new users
        -Repeat users: users which have transaction records
        -New users: users without transaction records
    Input:
        -User_balance: user_balance data frame
        -user_type: binary ['repeat', 'new']
    
    '''
    grouped = df.groupby(['user_id'])['report_date'].agg({'first_record':'min', 'last_record' :'max'})
    grouped['user_type'] = np.where(grouped[max]-grouped[min] != '0 days', 'repeat', 'new')
    
    
    if user_type == 'new':
        type_new = grouped[grouped['user_type'] == 'new'].reset_index()
        new_user = df.merge(type_new, on = 'user_id',how = 'inner')
        return new_user
    if user_type == 'repeat':
        type_repeat = grouped[grouped['user_type'] == 'repeat'].reset_index()
        repeat_user = df.merge(type_repeat, on = 'user_id',how = 'inner')
        return repeat_user
    
    

In [ ]:
def query_user_by_id(user_id, df = user_balance):
    '''
    query users by user id
    input: user balance dataframe, user_id (int or list)
    output: queried users
    '''
    
    try:
        queried = df.loc[df.user_id.isin(user_id)]

    except TypeError:
        queried = df.loc[df.user_id == user_id]
        if user_id not in df.user_id:
            raise ValueError('That is not an existing user!')
    
    return queried

In [ ]:
def miss_value_count(df):
    '''
    Return the missing values count for each column
    
    '''
    for col in df.columns:
        print("Missing value count for {}:".format(col), df[col].isnull().sum())

In [ ]:
user_type('new')

In [ ]:
query_user_by_id([2,3])

In [ ]:
miss_value_count(user_balance)

# Data Processing

In [ ]:
user_balance.head()

In [ ]:
user_balance.dtypes

In [ ]:
user_balance_ = user_balance.copy()
user_balance_.set_index('report_date', inplace = True)
# groupby(car_stream.index.day).mean()

In [ ]:

# = week_stream['report_date'].apply(lambda x: x.weekday())

In [ ]:
user_balance_ = user_balance.copy()
user_balance_.set_index('report_date', inplace = True)
user_balance_day = user_balance_.iloc[:,1:].groupby(user_balance_.index.day).sum()


fig, ax = plt.subplots(figsize = (12,5))
ax.plot(user_balance_day.index,user_balance_day['total_purchase_amt'], linestyle='-', linewidth=1, label='total_purchase_amt')
ax.plot(user_balance_day.index,user_balance_day['total_redeem_amt'], linestyle='-', linewidth=1, label='total_redeem_amt')
ax.set_xticks(user_balance_day.index)
plt.legend()
plt.xlabel('day of the month')
plt.ylabel('sum($)')
plt.title('Purchase/redemption by day')
plt.savefig('plot')
plt.tight_layout()

In [ ]:
user_balance_month = user_balance_.iloc[:,1:].groupby(user_balance_.index.month).mean()

fig, ax = plt.subplots(figsize = (12,5))
ax.plot(user_balance_month.index,user_balance_month['total_purchase_amt'], linestyle='-', linewidth=1, label='total_purchase_amt')
ax.plot(user_balance_month.index,user_balance_month['total_redeem_amt'], linestyle='-', linewidth=1, label='total_redeem_amt')
ax.set_xticks(user_balance_month.index)
plt.legend()
plt.xlabel('month')
plt.ylabel('sum($)')
plt.title('Purchase/redemption by month')
plt.savefig('plot')
plt.tight_layout()

In [ ]:
import calendar
user_balance_week = user_balance.copy()
# user_balance_week['start_time'] = week_stream.index
user_balance_week['day_of_week'] = user_balance_week['report_date'].apply(lambda x: x.weekday()) # get the weekday index, between 0 and 6
user_balance_week['day_of_week'] = user_balance_week['day_of_week'].apply(lambda x: calendar.day_name[x])
user_balance_week = user_balance_week.groupby('day_of_week').sum()


In [ ]:
sorter = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
sorterIndex = dict(zip(sorter,range(len(sorter))))
sorterIndex
user_balance_week['Day_id'] = user_balance_week.index
user_balance_week['Day_id'] = user_balance_week['Day_id'].map(sorterIndex)
user_balance_week.sort_values('Day_id', inplace=True)
user_balance_week.head()


In [ ]:
fig, ax = plt.subplots(figsize = (12,5))
ax.plot(user_balance_week.index,user_balance_week['total_purchase_amt'], linestyle='-', linewidth=1, label='total_purchase_amt')
ax.plot(user_balance_week.index,user_balance_week['total_redeem_amt'], linestyle='-', linewidth=1, label='total_redeem_amt')
ax.set_xticks(user_balance_week.index)
plt.legend()
plt.xlabel('weekdays')
plt.ylabel('sum($)')
plt.title('Purchase/redemption by month')
plt.savefig('plot')
plt.tight_layout()

In [ ]:
user_balance_group = user_balance_.copy()
user_balance_group.reset_index(drop = False,inplace = True)
user_balance_group.sort_values(['user_id','report_date'],ascending=True,inplace =True)

In [ ]:
user_balance_group.head()

In [ ]:
def last_item(df):
    return df.iloc[-1]

In [ ]:
columns = user_balance_group.iloc[:,2:].columns
func_dict = {}
for i in columns:
    if i == 'tBalance' or i == 'yBalance':
        func_dict[i] = last_item
    else:
        func_dict[i] = 'mean'
# func_dict.items()




user_balance_byID = user_balance_group.groupby('user_id').agg(func_dict)
user_balance_byID = user_balance_byID[columns]
user_balance_byID.fillna(0,inplace = True)
user_balance_byID.head()

# KMeans

In [ ]:
from sklearn import preprocessing
from sklearn.cluster import KMeans
columns = user_balance_byID.columns
user_balance_byID_sc = preprocessing.normalize(user_balance_byID)
user_balance_byID_sc = pd.DataFrame(user_balance_byID_sc, columns = columns)
user_balance_byID_sc.head()

In [ ]:
def report(n, cluster):
    print("{} clusters were formed.".format(n))
    for c in range(n):
        n_ele = sum([1 for x in cluster if x == c])
        print("Cluster {} has {} elements".format(c, n_ele))

In [ ]:
from sklearn.metrics import silhouette_score

range_n_clusters = range(4,8)
silhouette_avg = {}
results = {}

print("For clustering method KMeans:")
for n in range_n_clusters:
    km = KMeans(n_clusters=n, random_state=123)
    km.fit(user_balance_byID_sc)
    res = km.predict(user_balance_byID_sc)
    results[n] = res
    silhouette_avg[n] = silhouette_score(user_balance_byID_sc, res)

best_n = max(silhouette_avg.keys(), key=(lambda key: silhouette_avg[key]))
cluster = results[best_n]
report(best_n, cluster)

In [ ]:
user_balance_byID['label'] = cluster
user_balance_byID[user_balance_byID.label == 0].describe()

In [ ]:
user_balance_byID[user_balance_byID.label == 1].describe()

In [ ]:
user_balance_byID[user_balance_byID.label == 2].describe()

In [ ]:
user_balance_byID[user_balance_byID.label == 3].describe()

In [ ]:
user_balance_byID[user_balance_byID.label == 4].describe()

In [ ]:
user_balance_byID[user_balance_byID.label == 5].describe()

In [ ]:
user_balance_byID.head()

In [ ]:
key_array = zip(user_balance_byID.index, user_balance_byID.label)
cluster_dict = dict(key_array)

In [ ]:
def MatchClusterDict(user_ids, cluster_dict):
    cluster_list = []
    for user_id in user_ids:

        cluster = cluster_dict[user_id]

        cluster_list.append(cluster)
    return cluster_list

In [ ]:
cluster_list = np.array(MatchClusterDict(user_balance_.user_id, cluster_dict))
cluster_list

In [ ]:
user_balance_['user_cluster'] = cluster_list

In [ ]:
user_balance_[user_balance_['user_cluster'] == 4].head()

In [ ]:
user_balance_['date'] = user_balance_.index
user_balance_['day_of_week'] = user_balance_['date'].apply(lambda x: x.weekday()) # get the weekday index, between 0 and 6
user_balance_['day_of_week'] = user_balance_['day_of_week'].apply(lambda x: calendar.day_name[x])
# user_balance_.drop(['date'],inplace = True)


In [ ]:
user_balance_['month'] = user_balance_.index.month # get the weekday index, between 0 and 6
user_balance_['month'] = user_balance_['month'].apply(lambda x: calendar.month_name[x])

In [ ]:
user_balance_.head()

In [ ]:
weekday_dummy = pd.get_dummies(user_balance_['day_of_week'])
user_dummy = pd.get_dummies(user_balance_['user_cluster'])
month_dummy = pd.get_dummies(user_balance_['month'])

month_sorther = [calendar.month_name[x] for x in range(1,13)]
weekday_sorter = [calendar.day_name[x] for x in range(0,7)]

weekday_dummy = weekday_dummy[weekday_sorter]
month_dummy = month_dummy[month_sorther]

In [ ]:
# from functools import reduce

# dfs = [user_balance_, weekday_dummy, user_dummy, month_dummy]
# df_merged = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True), dfs)

df_merged = pd.merge(user_balance_,weekday_dummy,left_index=True, right_index=True)

In [ ]:
weekday_dummy.head()